In [6]:
import torch
from torch import nn
from torchvision.models import resnet50


class DETR(nn.Module):  # 定义DETR类，继承自nn.Module
    def __init__(self, num_classes, hidden_dim, nheads,
                 num_encoder_layers, num_decoder_layers):
        super().__init__()  # 调用基类的初始化方法
        # 使用ResNet-50模型的卷积层作为特征提取器，并去掉最后两层，最后两层是自适应池化和全连接层
        self.backbone = nn.Sequential(*list(resnet50(pretrained=True).children())[:-2])
        # 一个投射层，也就把这个 2048 变成256，也就是这层 conv投射层
        self.conv = nn.Conv2d(2048, hidden_dim, 1)  # 1x1卷积核，用于降维
        self.transformer = nn.Transformer(hidden_dim, nheads,
                                          num_encoder_layers, num_decoder_layers)  # Transformer模块
        self.linear_class = nn.Linear(hidden_dim, num_classes + 1)  # 分类器，输出类别数+1（背景），这个1代表背景
        self.linear_bbox = nn.Linear(hidden_dim, 4)  # 边界框回归器，4代表x,y,w,h
        self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))  # 可学习的查询位置参数,100个框，用做transformer的decoder的输入
        self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))  # 可学习的行嵌入参数 （50，hidden_dim/2）
        self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))  # 可学习的列嵌入参数 （50，hidden_dim/2）

    def forward(self, inputs):
        x = self.backbone(inputs)  # (N, C, H, W)--> (N, 2048, H/32, W/32) (1,2048,25,38)
        print(f'Retnet-50 backbone(骨干网络)输出的特征图的shape: {x.shape}')
        h = self.conv(x)  # (N, 2048, H/32, W/32)--> (N, hidden_dim, H/32, W/32),降维
        print(f'h.shape---{h.shape}')
        H, W = h.shape[-2:]  # (N, hidden_dim, H/32, W/32)--> (H/32, W/32) (25, 38)
        #将位置编码与特征图的展平版本相结合，并通过Transformer处理，得到预测结果，包括分类结果和边界框回归结果
        # self.col_embed[:W]：从 self.col_embed 中取出前 W 个元素，其中 W 是特征图的宽度。这将生成一个形状为 [W, hidden_dim // 2] 的矩阵，表示列的位置编码

        # self.row_embed[:H]：从 self.row_embed 中取出前 H 个元素，其中 H 是特征图的高度。这将生成一个形状为 [H, hidden_dim // 2] 的矩阵，表示行的位置编码
        pos = torch.cat([
            self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1), # (25, 38, hidden_dim/2)
            self.row_embed[:H].unsqueeze(1).repeat(1, W, 1), # (25, 38, hidden_dim/2)
        ], dim=-1)
        print(pos.shape) # (25，38, hidden_dim)
        pos = pos.flatten(0, 1).unsqueeze(1)  # (H/32, W/32, hidden_dim)--> (H*W, 1, hidden_dim)
        print(pos.shape) # (H*W, 1, hidden_dim) (950, 1, 256)
        #这里950输入相当于是序列，因为torch的transformer模块要求输入的维度是(S,N,E)，S是序列长度，N是batch大小，E是embedding维度，batch_first 如果为True,那么序列在最前面,flatten(2)代表最后两维展平
        h = self.transformer(pos + h.flatten(2).permute(2, 0, 1), #h展平permute(2,0,1),尺寸变为(950, 1, 256)
                             self.query_pos.unsqueeze(1))
        print(f'输出尺寸：{h.shape}') # (100, 1, 256)
        return self.linear_class(h), self.linear_bbox(h).sigmoid()

#这里91是coco数据集的类别数
detr = DETR(num_classes=91, hidden_dim=256, nheads=8,
            num_encoder_layers=6, num_decoder_layers=6)
detr.eval()
inputs = torch.randn(1, 3, 800, 1200) #高800宽1200的输入图像
logits, bboxes = detr(inputs)
logits.shape, bboxes.shape #输出92类+1（背景）的分类结果和边界框回归结果

Retnet-50 backbone(骨干网络)输出的特征图的shape: torch.Size([1, 2048, 25, 38])
h.shape---torch.Size([1, 256, 25, 38])
torch.Size([25, 38, 256])
torch.Size([950, 1, 256])
输出尺寸：torch.Size([100, 1, 256])


(torch.Size([100, 1, 92]), torch.Size([100, 1, 4]))

In [2]:
800/25

32.0